In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

2025-03-06 20:18:19.952442: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 20:18:19.965709: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741288699.984104   40059 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741288699.989143   40059 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 20:18:20.006801: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
## Load the trained model, scaler and encoders

model = load_model('model.keras')

with open('geo_enc.pkl','rb') as file:
    geo_enc = pickle.load(file)
    
with open('gen_enc.pkl','rb') as file:
    gen_enc = pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scalar = pickle.load(file)

I0000 00:00:1741288848.015253   40059 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1768 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
/home/jeswin/projects/tf_cuda/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
## Input sample
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [4]:
geo_encoded = geo_enc.transform([[input_data['Geography']]]).toarray()
geo_df = pd.DataFrame(geo_encoded, columns=geo_enc.get_feature_names_out(['Geography']))

/home/jeswin/projects/tf_cuda/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [5]:
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_data = pd.DataFrame([input_data])

In [7]:
input_data['Gender'] = gen_enc.transform(input_data['Gender'])

In [8]:
input_data = pd.concat([input_data.drop('Geography', axis=1), geo_df], axis=1)
input_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
input_data = scalar.transform(input_data)

In [10]:
prediction = model.predict(input_data)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step


I0000 00:00:1741289825.851068   40590 service.cc:148] XLA service 0x7fbc0c004310 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741289825.851125   40590 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-03-06 20:37:05.856555: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741289825.869905   40590 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1741289825.942209   40590 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


array([[0.03530978]], dtype=float32)

In [11]:
prediction_proba = prediction[0][0]
prediction_proba

np.float32(0.035309777)

In [12]:
if prediction_proba > 0.5:
    print("Customer is likely to CHURN.")
else:
    print("Customer is not likely to CHURN.")

Customer is not likely to CHURN.
